# Predicting Energy Consumption and Production Using Transformer Models

## Description
In this notebook, we will develop a machine learning model to predict whether the energy consumption of a household will be lower than its energy production in the next hour. We will use a Transformer model with the Time2Vector embedding to capture temporal information. The process includes data loading, preprocessing, normalization, PCA for dimensionality reduction, model training, and evaluation. Let's dive into the process step-by-step.

### Imports

We start by importing all necessary libraries and modules required for our analysis and model building.

In [62]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.saving import register_keras_serializable
import time
import joblib
import matplotlib.pyplot as plt
import logging
import coloredlogs


### Global Variables
We define some global constants that will be used throughout the notebook, such as sequence length, batch size, learning rate, and other model parameters.

In [63]:
# Global variables
SEQ_LENGTH = 60
TRAIN_END_DATE = '2023-12-31'
HEAD_SIZE = 256
NUM_HEADS = 4
FF_DIM = 4
NUM_TRANSFORMER_BLOCKS = 4
MLP_UNITS = [128]
DROPOUT = 0.1
MLP_DROPOUT = 0.1
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
EPOCHS = 1
EARLY_STOPPING_PATIENCE = 5

DATA_DIR = 'data'
MODELS_DIR = 'models'
PROCESSED_DATA_DIR = 'processed_data'

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

### Logging Configuration
We configure the logging to track the progress and debug information during the execution of the notebook.

In [64]:
# Logging configuration
logger = logging.getLogger(__name__)
coloredlogs.install(level='DEBUG', logger=logger, fmt='%(asctime)s %(levelname)s %(message)s')

# Ignore TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')


### TensorFlow and Device Configuration
We configure TensorFlow to use GPUs if available for faster computations.

In [65]:
# Forcing GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        logger.info(f"{len(gpus)} physical GPUs, {len(logical_gpus)} logical GPUs")
    except RuntimeError as e:
        logger.error(e)

2024-07-29 14:00:59 INFO 1 physical GPUs, 1 logical GPUs


### Function Definitions
We define all the functions required for data loading, preprocessing, PCA application, dataset creation, model building, and evaluation.

In [66]:


def load_data(file_path):
    logger.info("Loading data")
    return pd.read_csv(file_path, parse_dates=['cet_cest_timestamp'], index_col='cet_cest_timestamp')

def preprocess_data(data):
    logger.info("Preparing data")
    data['residential2_circulation_pump'] = data['DE_KN_residential2_circulation_pump'].diff().fillna(0)
    data['residential2_dishwasher'] = data['DE_KN_residential2_dishwasher'].diff().fillna(0)
    data['residential2_freezer'] = data['DE_KN_residential2_freezer'].diff().fillna(0)
    data['residential2_washing_machine'] = data['DE_KN_residential2_washing_machine'].diff().fillna(0)
    data['total_consumption'] = data['DE_KN_residential2_grid_import'].diff().fillna(0)
    data['total_production'] = data['DE_KN_residential1_pv']
    data = data.ffill()
    data['air_conditioning_on'] = 0
    data.loc[data['total_production'] > data['total_consumption'], 'air_conditioning_on'] = 1
    data['air_conditioning_on'] = data['air_conditioning_on'].shift(1, fill_value=0)
    X_minutes = 60
    data['target'] = data['air_conditioning_on'].rolling(window=X_minutes).sum().shift(-X_minutes) > X_minutes * 0.5
    data.dropna(subset=['target'], inplace=True)
    data['target'] = data['target'].astype(int)  # Convert target to integer
    return data

def select_features(data):
    logger.info("Selecting features")
    features = [
        'DE_KN_residential2_circulation_pump', 
        'DE_KN_residential2_dishwasher', 
        'DE_KN_residential2_freezer', 
        'DE_KN_residential2_grid_import',
        'DE_KN_residential2_washing_machine',
        'DE_KN_residential1_pv'
    ]
    return data[features], data['target']

def normalize_data(X_train, X_valid, X_test):
    logger.info("Normalizing data")
    scaler_path = os.path.join(MODELS_DIR, 'scaler.pkl')
    if os.path.exists(scaler_path):
        scaler = joblib.load(scaler_path)
        logger.info(f"Loading scaler from {scaler_path}")
    else:
        scaler = StandardScaler()
        scaler.fit(X_train)
        joblib.dump(scaler, scaler_path)
        logger.info(f"Saving scaler to {scaler_path}")
    X_train = scaler.transform(X_train)
    X_valid = scaler.transform(X_valid)
    X_test = scaler.transform(X_test)
    return X_train, X_valid, X_test

def apply_pca(X_train, X_valid, X_test):
    logger.info("Applying PCA")
    pca_path = os.path.join(MODELS_DIR, 'pca.pkl')
    if os.path.exists(pca_path):
        pca = joblib.load(pca_path)
        logger.info(f"PCA model loaded from {pca_path}")
    else:
        pca = PCA(n_components=0.95)  # Retain 95% of variance
        pca.fit(X_train)
        joblib.dump(pca, pca_path)
        logger.info(f"Saved PCA model to {pca_path}")
    X_train = pca.transform(X_train)
    X_valid = pca.transform(X_valid)
    X_test = pca.transform(X_test)
    return X_train, X_valid, X_test

def remove_nans(X, y):
    logger.info("Deleting rows with NaN values")
    mask = ~np.isnan(X).any(axis=1)
    X = X[mask]
    y = y[mask]
    return X, y


def create_tf_dataset(X, y, seq_length, batch_size, dataset_name):
    logger.info(f"Creating dataset {dataset_name}")
    X_data, y_data = [], []
    for i in range(len(X) - seq_length - 1):
        X_data.append(X[i:(i + seq_length)])
        y_data.append(y[i + seq_length])
    X_data, y_data = np.array(X_data), np.array(y_data)
    with tf.device('/gpu:0'):
        dataset = tf.data.Dataset.from_tensor_slices((X_data, y_data))
        dataset = dataset.cache().shuffle(buffer_size=len(X_data)).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

@register_keras_serializable()
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        final_output = self.layernorm2(out1 + ffn_output)
        return final_output

    def build(self, input_shape):
        super(TransformerBlock, self).build(input_shape)

    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'embed_dim': self.att.key_dim,
            'num_heads': self.att.num_heads,
            'ff_dim': self.ffn.layers[0].units,
            'rate': self.dropout1.rate,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

@register_keras_serializable()
class Time2Vector(layers.Layer):
    def __init__(self, seq_len, **kwargs):
        super(Time2Vector, self).__init__(**kwargs)
        self.seq_len = seq_len

    def build(self, input_shape):
        self.weights_linear = self.add_weight(name='weight_linear', shape=(int(self.seq_len),), initializer='uniform', trainable=True)
        self.bias_linear = self.add_weight(name='bias_linear', shape=(int(self.seq_len),), initializer='uniform', trainable=True)
        self.weights_periodic = self.add_weight(name='weight_periodic', shape=(int(self.seq_len),), initializer='uniform', trainable=True)
        self.bias_periodic = self.add_weight(name='bias_periodic', shape=(int(self.seq_len),), initializer='uniform', trainable=True)

    def call(self, x):
        x = tf.math.reduce_mean(x[:,:,:4], axis=-1)
        time_linear = self.weights_linear * x + self.bias_linear
        time_linear = tf.expand_dims(time_linear, axis=-1)
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        time_periodic = tf.expand_dims(time_periodic, axis=-1)
        return tf.concat([time_linear, time_periodic], axis=-1)

    def get_config(self):
        config = super(Time2Vector, self).get_config()
        config.update({'seq_len': self.seq_len})
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout, mlp_dropout):
    inputs = layers.Input(shape=input_shape)
    time_embedding = Time2Vector(SEQ_LENGTH)(inputs)
    x = layers.Concatenate(axis=-1)([inputs, time_embedding])
    x = layers.Dense(head_size, dtype='float32')(x)
    for _ in range(num_transformer_blocks):
        x = TransformerBlock(head_size, num_heads, ff_dim, dropout)(x, training=True)
    x = layers.GlobalAveragePooling1D()(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return Model(inputs, outputs)

def plot_loss(history):
    logger.info("Plotting training and validation loss")
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()


### Load Data
We load the household energy consumption data from a CSV file.

In [67]:
# Load data
data = load_data(os.path.join(DATA_DIR, 'household_data_1min_singleindex.csv'))

2024-07-29 14:00:59 INFO Loading data
/var/folders/gd/q32w8wgd1bxc7wq53vd9nts80000gn/T/ipykernel_940/992125112.py:3: DtypeWarning: Columns (70) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, parse_dates=['cet_cest_timestamp'], index_col='cet_cest_timestamp')


### Data Preprocessing
We preprocess the data by creating new features, handling missing values, and defining the target variable.

In [68]:
# Preprocess data
data_preprocessed = preprocess_data(data)
X, y = select_features(data_preprocessed)

2024-07-29 14:01:27 INFO Preparing data
2024-07-29 14:01:32 INFO Selecting features


### Split Data into Training, Validation, and Test Sets
We split the data into training, validation, and test sets to prepare for model training and evaluation.

In [69]:
# Divide data into train, validation and test sets
split_1 = int(len(X) * 0.7)
split_2 = int(len(X) * 0.85)
X_train, X_valid, X_test = X[:split_1], X[split_1:split_2], X[split_2:]
y_train, y_valid, y_test = y[:split_1], y[split_1:split_2], y[split_2:]

### Remove NaNs
We remove any rows containing NaN values from the datasets before applying PCA.

In [70]:
# Delete rows with NaN values before normalization
X_train, y_train = remove_nans(X_train, y_train)
X_valid, y_valid = remove_nans(X_valid, y_valid)
X_test, y_test = remove_nans(X_test, y_test)

2024-07-29 14:01:32 INFO Deleting rows with NaN values
2024-07-29 14:01:32 INFO Deleting rows with NaN values
2024-07-29 14:01:32 INFO Deleting rows with NaN values


### Data Normalization
We normalize the data to ensure that all features have the same scale.

In [71]:
# Normalize data
X_train_norm, X_valid_norm, X_test_norm = normalize_data(X_train, X_valid, X_test)

2024-07-29 14:01:33 INFO Normalizing data
2024-07-29 14:01:33 INFO Loading scaler from models/scaler.pkl


### Apply PCA
We apply PCA to reduce the dimensionality of the data, retaining 95% of the variance.

In [72]:

# Apply PCA
X_train_pca, X_valid_pca, X_test_pca = apply_pca(X_train_norm, X_valid_norm, X_test_norm)

2024-07-29 14:01:33 INFO Applying PCA
2024-07-29 14:01:33 INFO PCA model loaded from models/pca.pkl


### Create TensorFlow Datasets
We create TensorFlow datasets from the processed and normalized data, which will be used for training and evaluating the model.

In [73]:
# Create datasets
train_dataset = create_tf_dataset(X_train_pca, y_train, SEQ_LENGTH, BATCH_SIZE, 'train')
valid_dataset = create_tf_dataset(X_valid_pca, y_valid, SEQ_LENGTH, BATCH_SIZE, 'valid')
test_dataset = create_tf_dataset(X_test_pca, y_test, SEQ_LENGTH, BATCH_SIZE, 'test')

2024-07-29 14:01:33 INFO Creating dataset train
/var/folders/gd/q32w8wgd1bxc7wq53vd9nts80000gn/T/ipykernel_940/992125112.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_data.append(y[i + seq_length])
2024-07-29 14:01:41 INFO Creating dataset valid
2024-07-29 14:01:42 INFO Creating dataset test


### Model Training
We build, compile, and train the Transformer model using the training and validation datasets.

In [74]:

model_path = os.path.join(MODELS_DIR, 'transformer_model.keras')
if os.path.exists(model_path):
    logger.info("Loading existing model")
    model = tf.keras.models.load_model(model_path, custom_objects={"Time2Vector": Time2Vector, "TransformerBlock": TransformerBlock})
else:
    input_shape = (SEQ_LENGTH, X_train_pca.shape[1])
    logger.info("Building new model")
    model = build_model(input_shape, HEAD_SIZE, NUM_HEADS, FF_DIM, NUM_TRANSFORMER_BLOCKS, MLP_UNITS, DROPOUT, MLP_DROPOUT)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss="binary_crossentropy", metrics=["accuracy"])

model.summary()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, restore_best_weights=True)

start_time = time.time()

history = model.fit(train_dataset, validation_data=valid_dataset, epochs=EPOCHS, callbacks=[early_stopping], verbose=1)

end_time = time.time()
training_time = end_time - start_time
logger.info(f"Training time: {training_time:.2f} seconds")

model.save(model_path)
logger.info(f"Model saved to {model_path}")

2024-07-29 14:01:44 INFO Loading existing model


Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15      │ (None, 60, 2)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time2_vector_3      │ (None, 60, 2)     │        240 │ input_layer_15[0… │
│ (Time2Vector)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 60, 4)     │          0 │ input_layer_15[0… │
│ (Concatenate)       │                   │            │ time2_vector_3[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_33 (Dense)    │ (None, 60, 256)   │      1,280 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_… │ (None, 60, 256)   │  1,055,236 │ dense_33[0][0]    │
│ (TransformerBlock)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_… │ (None, 60, 256)   │  1,055,236 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_… │ (None, 60, 256)   │  1,055,236 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_block_… │ (None, 60, 256)   │  1,055,236 │ transformer_bloc… │
│ (TransformerBlock)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ transformer_bloc… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 128)       │     32,896 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_51          │ (None, 128)       │          0 │ dense_42[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 1)         │        129 │ dropout_51[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,255,489 (16.23 MB)

 Trainable params: 4,255,489 (16.23 MB)

 Non-trainable params: 0 (0.00 B)

43226/43226 ━━━━━━━━━━━━━━━━━━━━ 5764s 133ms/step - accuracy: 1.0000 - loss: 7.6679e-11 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


2024-07-29 15:37:49 INFO Training time: 5763.91 seconds
2024-07-29 15:37:50 INFO Model saved to models/transformer_model.keras


### Model Evaluation
We evaluate the model using the test dataset and visualize the training and validation loss.

In [75]:

logger.info("Evaluation on test set")
test_loss, test_accuracy = model.evaluate(test_dataset)
logger.info(f"Loss on test: {test_loss}")
logger.info(f"Accuracy on test: {test_accuracy}")

#plot_loss(history)


2024-07-29 15:37:50 INFO Evaluation on test set


10813/10813 ━━━━━━━━━━━━━━━━━━━━ 182s 17ms/step - accuracy: 0.9998 - loss: 0.0044


2024-07-29 15:40:52 INFO Loss on test: 0.004827520810067654
2024-07-29 15:40:52 INFO Accuracy on test: 0.9998294711112976
